In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"
igfold = 4
os.system('mkdir -p ./data_folds/fold_' + str(igfold))

0

In [2]:
!nvidia-smi

Fri Jan 13 11:53:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   42C    P0    73W / 163W |   1172MiB / 32510MiB |     75%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [3]:
import cudf
import glob
import gc

from tqdm import tqdm

In [4]:
lb = False
path = '../data/'
type_weight = {0:1, 1:6, 2:3}
no_files = 5
path = '../../data/'
    
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [5]:
# !rm -rf ./data/
!mkdir -p ./data/

In [6]:
files = sorted(
    glob.glob('../../data/' + '/train/interim/*.parquet')
)
files_split = [glob.glob('../../data/test.parquet')] + [glob.glob('./data/xgb_train_x.parquet')] + list_in_chunks(files, no_chunks=len(files)//no_files)

In [7]:
import pickle

sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
if igfold == 0:
    sess_eval = sessions[0]+sessions[1]
elif igfold == 1:
    sess_eval = sessions[2]+sessions[3]
elif igfold == 2:
    sess_eval = sessions[4]+sessions[5]
elif igfold == 3:
    sess_eval = sessions[6]+sessions[7]
elif igfold == 4:
    sess_eval = sessions[8]+sessions[9]
print(len(sess_eval))

356746


In [8]:
len(files_split)

27

### 1) "Carts Orders" Co-visitation Matrix - Type Weighted

In [9]:
!rm -r ./data/tmp
!mkdir -p ./data/tmp/split

In [10]:
out = []
for e, file in tqdm(enumerate(files_split)):
    df = cudf.read_parquet(file)
    df = df.merge(
        df_type,
        how='left',
        on='type'
    )
    if any([True for x in file if 'xgb_train_x.parquet' in x]):
        print(file)
        df = df[df['session'].isin(sess_eval)]
    else:
        df = df.loc[~(df['session'].isin(sess_eval))]

    df['session'] = df['session'].astype('int32')
    df['aid'] = df['aid'].astype('int32')
    df.ts = (df.ts/1000).astype('int32')
    df.drop(['type'], axis=1, inplace=True)
    df = df.rename(columns={'type_': 'type'})
    df = df.sort_values(['session','ts'],ascending=[True,False])
    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)
    df = df.merge(
        df, 
        how='left',
        on='session'
    )
    gc.collect()
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]
    gc.collect()
    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    gc.collect()
    df['wgt'] = df.type_y.map(type_weight)
    df = df[['aid_x','aid_y','wgt']]
    gc.collect()
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum().reset_index()
    df.to_parquet('./data/tmp/split/split_' + str(e) + '.parquet')
    del df
    gc.collect()

1it [00:02,  2.61s/it]

['./data/xgb_train_x.parquet']


27it [00:59,  2.20s/it]


In [11]:
files_split1 = glob.glob('./data/tmp/split/split_*.parquet')
df = cudf.read_parquet(files_split1[0])
for iifile, file in enumerate(files_split1[1:]):
    print(file)
    df2 = cudf.read_parquet(file)
    df = cudf.concat([
        df,
        df2
    ])
    gc.collect()
    df = df.groupby(['aid_x','aid_y']).sum()
    df = df.reset_index()
    del df2
    gc.collect()

./data/tmp/split/split_4.parquet
./data/tmp/split/split_11.parquet
./data/tmp/split/split_3.parquet
./data/tmp/split/split_26.parquet
./data/tmp/split/split_20.parquet
./data/tmp/split/split_25.parquet
./data/tmp/split/split_24.parquet
./data/tmp/split/split_1.parquet
./data/tmp/split/split_21.parquet
./data/tmp/split/split_2.parquet
./data/tmp/split/split_7.parquet
./data/tmp/split/split_0.parquet
./data/tmp/split/split_9.parquet
./data/tmp/split/split_18.parquet
./data/tmp/split/split_10.parquet
./data/tmp/split/split_8.parquet
./data/tmp/split/split_12.parquet
./data/tmp/split/split_23.parquet
./data/tmp/split/split_13.parquet
./data/tmp/split/split_6.parquet
./data/tmp/split/split_19.parquet
./data/tmp/split/split_17.parquet
./data/tmp/split/split_14.parquet
./data/tmp/split/split_5.parquet
./data/tmp/split/split_22.parquet
./data/tmp/split/split_16.parquet


In [12]:
!nvidia-smi

Fri Jan 13 11:55:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   41C    P0    47W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [13]:
gc.collect()
df = df.sort_values(['aid_x','wgt'],ascending=[True,False])
df = df.reset_index(drop=True)
gc.collect()


0

In [14]:
df['n'] = df.groupby('aid_x').aid_y.cumcount()

In [15]:
df = df.loc[df.n<50]
df.to_parquet('./data_folds/fold_' + str(igfold) + '/top_15_carts_orders_v3.parquet')

In [16]:
del df
gc.collect()

48

### Generate Candidates

In [17]:
!nvidia-smi

Fri Jan 13 11:55:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   41C    P0    61W / 163W |    796MiB / 32510MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [18]:
import cudf
import numpy as np
import gc
import os

import glob

In [19]:
def get_candidates(df, co, prefix, treshhold=2, treshhold_topn=15):
    df = df.merge(
        co[co['n']<treshhold_topn][['aid', 'cand', 'wgt']],
        how='left',
        on='aid'
    )
    df['wgt2'] = df['wgt']*df['score']
    df['wgt_cl'] = (df['type']=='clicks').astype('int8')*df['wgt']
    df['wgt_ca'] = (df['type']=='carts').astype('int8')*df['wgt']
    df['wgt_or'] = (df['type']=='orders').astype('int8')*df['wgt']
    df.drop(['aid', 'ts', 'type'], axis=1, inplace=True)
    df = df.groupby(['session', 'cand']).agg({
        'dummy': 'sum',
        'rank': 'min',
        'wgt': 'sum',
        'wgt2': 'sum',
        'wgt_cl': 'sum',
        'wgt_ca': 'sum',
        'wgt_or': 'sum'
    }).reset_index()
    df.columns = [
        'session', 'cand', 
        prefix+'_num_' + str(treshhold_topn),
        prefix+'_rank_min_'+ str(treshhold_topn), 
        prefix+'_wgt_' + str(treshhold_topn),
        prefix+'_wgt_2_'+ str(treshhold_topn), 
        prefix+'_wgt_cl_'+ str(treshhold_topn), 
        prefix+'_wgt_ca_'+ str(treshhold_topn), 
        prefix+'_wgt_or_'+ str(treshhold_topn)
    ]
    return(df)

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [20]:
prefix = 'caco'

In [21]:
for dataset in ['train', 'sub']:
    for treshhold_topn in [15]:
        print(dataset, treshhold_topn)
        os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn))
        co = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/top_15_carts_orders_v3.parquet')
        co.columns = ['aid', 'cand', 'wgt', 'n']
        if dataset == 'train':
            df = cudf.read_parquet('./data/xgb_train_x.parquet')
            df = df[df['session'].isin(sess_eval)]
        else:
            df = cudf.read_parquet('../../data/test.parquet')
        df = df.sort_values(['session', 'ts'], ascending=[True, False])
        df['dummy'] = 1
        df['rank'] = df.groupby(['session']).dummy.cumsum()
        df['score'] = 1/df['rank']
        session_lists = list_in_chunks(df['session'].drop_duplicates().to_pandas().values.tolist(), no_chunks=10)
        out = []
        for session_list in session_lists:
            df_tmp = df[df['session'].isin(session_list)]
            df_tmp = get_candidates(df_tmp, co, prefix, treshhold=2, treshhold_topn=treshhold_topn)
            out.append(df_tmp)
        df = cudf.concat(out)
        df['session'] = df['session'].astype('int32')
        df['cand'] = df['cand'].astype('int32')
        df[prefix + '_num_'+ str(treshhold_topn)] = df[prefix + '_num_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_rank_min_'+ str(treshhold_topn)] = df[prefix + '_rank_min_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_'+ str(treshhold_topn)] = df[prefix + '_wgt_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_2_'+ str(treshhold_topn)] = df[prefix + '_wgt_2_'+ str(treshhold_topn)].astype('float32')
        df[prefix + '_wgt_cl_'+ str(treshhold_topn)] = df[prefix + '_wgt_cl_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_ca_'+ str(treshhold_topn)] = df[prefix + '_wgt_ca_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_or_'+ str(treshhold_topn)] = df[prefix + '_wgt_or_'+ str(treshhold_topn)].astype('int16')
        df.to_parquet('./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn) + '/cand.parquet')
        del df
        gc.collect()

train 15
sub 15
